In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# --- Definir Transformaciones ---
# 1. transforms.ToTensor(): Convierte la imagen (PIL) a un Tensor de PyTorch 
#    y escala los valores de los píxeles de [0, 255] a [0.0, 1.0].
# 2. transforms.Normalize(): Normaliza el tensor. (0.5) es la media 
#    y (0.5) la desviación estándar que usaremos para centrar los datos.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# --- Descargar y Cargar Datos ---
# PyTorch facilita esto inmensamente.
# train=True descarga el set de entrenamiento.
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# --- Crear los DataLoaders ---
# DataLoader es una herramienta increíblemente potente.
# Maneja por nosotros:
# 1. Batching: Agrupa los datos en "lotes" (ej. 64 imágenes a la vez).
# 2. Shuffling: Mezcla los datos en cada epoch para mejorar el aprendizaje.
# 3. Paralelismo: Carga los datos en hilos separados (num_workers).
batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100.0%
100.0%
100.0%
100.0%


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # --- Bloque Convolucional 1 ---
        # Entrada: [1, 28, 28] (1 canal de color)
        # Salida: [16, 28, 28] -> [16, 14, 14] (después de MaxPool)
        self.conv_block1 = nn.Sequential(
            # 1 canal de entrada, 16 "filtros" de salida, tamaño de filtro 3x3, padding=1 (para mantener el 28x28)
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # Reduce el tamaño a la mitad (28x28 -> 14x14)
        )
        
        # --- Bloque Convolucional 2 ---
        # Entrada: [16, 14, 14]
        # Salida: [32, 14, 14] -> [32, 7, 7] (después de MaxPool)
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # Reduce el tamaño a la mitad (14x14 -> 7x7)
        )
        
        # --- Capa Completamente Conectada (Clasificador) ---
        # Ahora "aplanamos" (flatten) la salida del bloque 2.
        # Tamaño de salida: 32 canales * 7 alto * 7 ancho = 1568 características
        # La entrada a la capa lineal DEBE ser 1568.
        # Salida final: 10 neuronas (una para cada dígito: 0, 1, ..., 9)
        self.fc_layer = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        # 1. Pasa por el primer bloque convolucional
        x = self.conv_block1(x)
        # 2. Pasa por el segundo bloque convolucional
        x = self.conv_block2(x)
        
        # 3. "Aplanar" el tensor para la capa lineal
        # Mantiene el tamaño del lote (-1) y aplana todo lo demás
        x = x.view(-1, 32 * 7 * 7) 
        
        # 4. Pasa por la capa clasificadora
        x = self.fc_layer(x)
        # NOTA: No aplicamos Softmax aquí porque la función de pérdida
        # (CrossEntropyLoss) lo hace internamente por nosotros.
        return x

# Instanciamos el modelo
model = CNN()

# (Opcional) Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [4]:
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
n_epochs = 5 # MNIST aprende rápido

for epoch in range(n_epochs):
    # 'train_loader' nos da lotes de (imágenes, etiquetas)
    for i, (images, labels) in enumerate(train_loader):
        
        # Mover los datos a la GPU (si la estamos usando)
        images = images.to(device)
        labels = labels.to(device)
        
        # 1. Forward pass
        outputs = model(images)
        
        # 2. Calcular Loss
        loss = loss_fn(outputs, labels)
        
        # 3. Resetear gradientes
        optimizer.zero_grad()
        
        # 4. Backward pass (Autograd)
        loss.backward()
        
        # 5. Actualizar pesos
        optimizer.step()
        
        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Lote [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/5], Lote [200/938], Loss: 0.1326
Epoch [1/5], Lote [400/938], Loss: 0.1699
Epoch [1/5], Lote [600/938], Loss: 0.0635
Epoch [1/5], Lote [800/938], Loss: 0.0326
Epoch [2/5], Lote [200/938], Loss: 0.0726
Epoch [2/5], Lote [400/938], Loss: 0.0270
Epoch [2/5], Lote [600/938], Loss: 0.0959
Epoch [2/5], Lote [800/938], Loss: 0.0424
Epoch [3/5], Lote [200/938], Loss: 0.0162
Epoch [3/5], Lote [400/938], Loss: 0.0372
Epoch [3/5], Lote [600/938], Loss: 0.0287
Epoch [3/5], Lote [800/938], Loss: 0.0652
Epoch [4/5], Lote [200/938], Loss: 0.0073
Epoch [4/5], Lote [400/938], Loss: 0.0022
Epoch [4/5], Lote [600/938], Loss: 0.0034
Epoch [4/5], Lote [800/938], Loss: 0.0645
Epoch [5/5], Lote [200/938], Loss: 0.1169
Epoch [5/5], Lote [400/938], Loss: 0.0694
Epoch [5/5], Lote [600/938], Loss: 0.0437
Epoch [5/5], Lote [800/938], Loss: 0.0056


In [6]:
# Poner el modelo en modo evaluación
model.eval()

# No necesitamos calcular gradientes durante la evaluación
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Hacemos la predicción
        outputs = model(images)
        
        # torch.max devuelve (valor_maximo, indice_maximo)
        # Nos interesa el índice (la clase predicha)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'\n--- Evaluación en el conjunto de Prueba ---')
    print(f'Precisión en las 10,000 imágenes de prueba: {accuracy:.2f}%')


--- Evaluación en el conjunto de Prueba ---
Precisión en las 10,000 imágenes de prueba: 99.02%
